In [1]:
print("PREPARANDO EL ENTORNO\n\n")
import os
 # Instalar SDK java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Descargar Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
# Descomprimir la versión de Spark
!tar xf spark-3.3.1-bin-hadoop3.tgz
# Establecer las variables de entorno
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"
# Descargar findspark
!pip install -q findspark
# Descargar pyspark
!pip install -q pyspark
print("\n\n******** INSTALACIÓN CORRECTA *******")

PREPARANDO EL ENTORNO


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.4 MB/s eta 0:00:00


******** INSTALACIÓN CORRECTA *******


In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

In [3]:
rdd = sc.parallelize([item for item in range(10)])

**Almacenamiento en caché**

In [4]:
from pyspark.storagelevel import StorageLevel

In [5]:
rdd.persist(StorageLevel.MEMORY_ONLY)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [6]:
# Para cambiar la persistencia hay que liberar la anterior
rdd.unpersist()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [7]:
rdd.persist(StorageLevel.DISK_ONLY)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [8]:
rdd.unpersist()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [9]:
# cache() es lo mismo que MEMORY_ONLY
rdd.cache()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [10]:
rdd.unpersist()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

**Particionado**

In [11]:
rdd = sc.parallelize(['x', 'y', 'z'])

In [12]:
hola = 'Hola'

In [13]:
hash(hola)

3356828804810898397

In [14]:
num_particiones = 6

In [15]:
# indice = hash(item) % num_particiones

In [16]:
hash('x') % num_particiones

4

In [17]:
hash('y') % num_particiones

2

In [18]:
hash('z') % num_particiones

3

**Mezcla de datos (Shuffling)**

##Broadcast variables**

In [19]:
rdd = sc.parallelize([item for item in range(10)])

In [20]:
uno = 1

In [21]:
br_uno = sc.broadcast(uno)

In [22]:
rdd1 = rdd.map(lambda x: x + br_uno.value)

In [23]:
rdd1.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [24]:
# Liberamos broadcast
rdd1.unpersist()

PythonRDD[3] at collect at <ipython-input-23-75ecb6e3744c>:1

In [25]:
# Se libera el broadcast, pero no se elimina, se puede volver a utilizar
rdd1 = rdd.map(lambda x: x + br_uno.value)

In [26]:
rdd1.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [27]:
# Para eliminarlo por completo
br_uno.destroy()

In [ ]:
# Esto va a arrojar un error, ya que la variable broadcast se elimino anteriormente
rdd1 = rdd.map(lambda x: x + br_uno.value)
rdd1.take(5)

**Acumuladores**

In [29]:
# Declaramos un acumulador
acumulador = sc.accumulator(0)

In [30]:
rdd = sc.parallelize([2, 4, 6, 8, 10])

In [31]:
from pyspark import accumulators
from pyspark.accumulators import Accumulator
# Add es como sumar
rdd.foreach(lambda x: acumulador.add(x))
print(acumulador.value)

30


In [32]:
rdd1 = sc.parallelize('Mi nombre es José Miguel y me siento genial'.split(' '))

In [33]:
acumulador1 = sc.accumulator(0)
rdd1.foreach(lambda x: acumulador1.add(1))
print(acumulador1.value)

9


**Ejercicio**

1. Cree un RDD importes a partir de los datos adjuntos a esta lección como recurso. Emplee acumuladores para obtener el total de ventas realizadas y el importe total de las ventas.

In [36]:
# Cargamos el fichero
importes = sc.textFile("./rdd.txt")
importes = importes.map(lambda _: int(_))
importes.take(5)

[527, 386, 701, 240, 941]

In [43]:
acumulador = sc.accumulator(0)
importes.foreach(lambda x: acumulador.add(1))
print(acumulador.value)

10000


In [44]:
importes.foreach(lambda x: acumulador.add(x))
print(acumulador.value)

5052335


2. Si se conoce que a cada venta hay que restarle un importe fijo igual a 10 pesos por temas de impuestos.

2.1. ¿Cómo restaría este impuesto de cada venta utilizando una variable broadcast para acelerar el proceso?

R. broadcast = 10, se resta en el lambda

2.2. Cree un RDD llamado ventas_sin_impuestos a partir de la propuesta del inciso a que contenga las ventas sin impuestos.

In [47]:
dies = 10
br_dies = sc.broadcast(dies)
ventas_sin_inpuestos = importes.map(lambda x: x - br_dies.value)
ventas_sin_inpuestos.take(5)

[517, 376, 691, 230, 931]

2.3. Destruya la variable broadcast creada luego de emplearla para crear el RDD del inciso b.

In [48]:
br_dies.destroy()

3. Persista el RDD ventas_sin_impuestos en los siguientes niveles de persistencia.

In [57]:
from pyspark.storagelevel import StorageLevel

In [58]:
ventas_sin_inpuestos.persist(StorageLevel.MEMORY_ONLY)

PythonRDD[30] at RDD at PythonRDD.scala:53

In [59]:
ventas_sin_inpuestos.unpersist()

PythonRDD[30] at RDD at PythonRDD.scala:53

In [60]:
ventas_sin_inpuestos.persist(StorageLevel.DISK_ONLY)

PythonRDD[30] at RDD at PythonRDD.scala:53

In [61]:
ventas_sin_inpuestos.unpersist()

PythonRDD[30] at RDD at PythonRDD.scala:53

In [62]:
ventas_sin_inpuestos.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[30] at RDD at PythonRDD.scala:53

In [63]:
ventas_sin_inpuestos.unpersist()

PythonRDD[30] at RDD at PythonRDD.scala:53